In [78]:
# Module Imports

#DB stuff
import mariadb
import sys

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time

In [79]:
#Connect with DB
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [80]:
########################
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]

def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

def pad(data):
  length = 16 - (len(data) % 16)
  return data + chr(length).encode('utf-8')*length

def encodeMessage(msg):
  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Erster Schritt: AES256 verschlüsseln
  iv = Random.new().read(AES.block_size)
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
  msg = msg.encode('utf-8')
  crypt = iv + cipher.encrypt(pad(msg))

  # Zweiter Schritt: base64 enkodieren
  msgb64 = base64.b64encode(crypt).decode('utf-8')
  return msgb64

#Gibt ein JSON-Objekt über die verfügbaren Daten zurück
def getAvailable():
    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Daten anfordern
    connection.request("GET", "/dirbuff")
    contents = connection.getresponse()

    # Daten entschlüsseln
    msg = decodeMessage(contents.read())
  
    #JSON-String umwandeln in dict
    folder_dict = json.loads(msg)
    
    # Verbindung trennen
    connection.close()

    return folder_dict


In [81]:
def file_to_db():
    folder = getAvailable()

    # Verbindung zum air-Q aufbauen
    headers = {'Content-type': 'Test'}
    connection = http.client.HTTPConnection(airqIP)

    # Anfrage formulieren und Daten anfordern
    connection.request("GET","/file?request="+encodeMessage("2023/3/27/1679878517"))
    contents = connection.getresponse()
    print(contents)
    # Daten zeilenweise entschlüsseln und in Datei "1589256960.txt" ausgeben
    with open("1679878517.txt","x") as f:
        for line in contents.read().split(b'\n'):
            if line != b'':
                f.write(decodeMessage(line)+'\n')

    # Verbindung trennen
    connection.close()


#Diese Daten beim einlesen rausfiltern

#bat -> rausfiltern
#DeviceID -> rausfiltern
#uptime-> rausfiltern
#window_event 
#door_event
#person
#window_open

In [82]:
file_to_db()

In [83]:
#DB-Connection close
conn.close()